## The objective of this problem is to explore different neighbourhoods in the city TORONTO

###### METHOD: We will be using the wickipedia page to extract the data through web scraping techniques with the help of Python requests and beautifulSoup packages. Then,we can get the latitude and longitude coordinates using Python Geocoder Package and we can also map it using folium .  After that,we will use the Foursquare API to explore the neighborhoods.

### Scraping data from wikipedia 

In [6]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
import folium
from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
pd.set_option('display.max_columns', None) #to display all the rows of table
pd.set_option('display.max_rows', None)

print("Imported!")

Imported!


In [7]:
!pip install geocoder

You should consider upgrading via the 'c:\python39\python.exe -m pip install --upgrade pip' command.



#### Scraping list of postal codes of toronto using BeautifulSoup

In [8]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data = requests.get(url).text
scrape_data = BeautifulSoup(data, 'lxml')

In [9]:
print(scrape_data.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"a04ee95d-1527-4a7b-b892-9d01efd10049","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":1032600019,"wgRevisionId":1032600019,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Communica

In [13]:
column_names=['Postalcode','Borough','Neighborhood']
toronto=pd.DataFrame(columns=column_names)

content=scrape_data.find('div',class_='mw-parser-output')
table=content.table.tbody
postcode=0
borough=0
neighborhood=0

for td in table.find_all('td'):
    postcode=td.p.text[0:3]
    spanTag=td.span
    i=0
    if spanTag.text=="Not assigned":
            pass
    else:    
        for sp in spanTag.find_all('a'):
            if i==0:
                borough=sp.text
                i+=1
            elif i==1:
                neighborhood=sp.text
        toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)            
    

In [14]:
toronto

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M7A,Queen's Park,Lawrence Heights
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Woodbine Gardens
9,M5B,Downtown Toronto,Ryerson


##### Cleaning dataframe

In [15]:
toronto=toronto[toronto.Borough!="Not assigned"]
toronto=toronto[toronto.Borough!=0]
toronto.reset_index(drop=True,inplace=True)

for i in range(toronto.shape[0]):
    if toronto.iloc[i][2]=='Not assigned':
        toronto.iloc[i][2]=toronto.iloc[i][1]
        

In [16]:
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
pd.set_option('display.max_rows',None)
df

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,Rouge
1,M1C,Scarborough,Highland Creek
2,M1E,Scarborough,West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,Birchmount Park
7,M1L,Scarborough,Oakridge
8,M1M,Scarborough,Scarborough Village
9,M1N,Scarborough,Cliffside


In [17]:
df.shape

(103, 3)

In [18]:
df.describe()

,Postalcode,Borough,Neighborhood
count,103,103,103
unique,103,25,78
top,M3H,North York,Downsview
freq,1,25,7


In [19]:
import geocoder # import geocoder
df1=df['Postalcode']
df_coords=pd.DataFrame(columns=['Latitude','Longitude'])    

for i in df1.iteritems():
# initialize your variable to None
    lat_lng_coords = None

# loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(i))
        lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    
    df['Latitude']=latitude
    df['Longitude']=longitude


In [20]:
df

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Rouge,43.71174,-79.57941
1,M1C,Scarborough,Highland Creek,43.71174,-79.57941
2,M1E,Scarborough,West Hill,43.71174,-79.57941
3,M1G,Scarborough,Woburn,43.71174,-79.57941
4,M1H,Scarborough,Cedarbrae,43.71174,-79.57941
5,M1J,Scarborough,Scarborough Village,43.71174,-79.57941
6,M1K,Scarborough,Birchmount Park,43.71174,-79.57941
7,M1L,Scarborough,Oakridge,43.71174,-79.57941
8,M1M,Scarborough,Scarborough Village,43.71174,-79.57941
9,M1N,Scarborough,Cliffside,43.71174,-79.57941


### Creating map 


In [21]:
!pip install geopy
!pip install folium

You should consider upgrading via the 'c:\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\python39\python.exe -m pip install --upgrade pip' command.


In [22]:
address= 'Toronto, Ontario Canada'
geolocator= Nominatim(user_agent="toronto-explorer")
location=geolocator.geocode(address)
latitude=location.latitude
longitude= location.longitude
print("The coordinates of the Toronto are {},{} .".format(latitude,longitude))

The coordinates of the Toronto are 43.6534817,-79.3839347 .


In [23]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)
map_toronto

### Selecting only the boroughs that contain the word TORONTO and will create a map again

In [24]:
df_borough=df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_borough.shape
df_borough.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M4R,North Toronto,Willowdale,43.71174,-79.57941
1,M4W,Downtown Toronto,Rosedale,43.71174,-79.57941
2,M4X,Downtown Toronto,Cabbagetown,43.71174,-79.57941
3,M4Y,Downtown Toronto,Church and Wellesley,43.71174,-79.57941
4,M5A,Downtown Toronto,Harbourfront,43.71174,-79.57941


In [25]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, borough, neighborhood in zip(df_borough['Latitude'], df_borough['Longitude'], df_borough['Borough'], df_borough['Neighborhood']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='red',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)
map_toronto

#### Using the Foursquare API to explore the neighbours

In [26]:
CLIENT_ID = 'ROWO3FEENYIKSG5PLO04KNTQPOAWGOCSWZ1XSJADC1MOIMHY' # Put Your Client Id
CLIENT_SECRET = 'BELNUT4PTLW5D0B50RTTQG00F2XAMWMFQIHGYOIC03TJNTEV' # Put You Client Secret 
VERSION = '20210707'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: Hidden')
print('CLIENT_SECRET: Hidden')

Your credentails:
CLIENT_ID: Hidden
CLIENT_SECRET: Hidden


### Exploring all venues by neighbourhoods and boroughs

In [27]:
def getNearbyVenues(boroughs, neighborhoods, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for borough, neighborhood, lat, lng in zip(boroughs, neighborhoods, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            borough,
            neighborhood, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
                  'Borough',
                  'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
borough_neighborhood_venues = getNearbyVenues(boroughs=df_borough['Borough'],
                                             neighborhoods=df_borough['Neighborhood'],
                                             latitudes=df_borough['Latitude'],
                                             longitudes=df_borough['Longitude'])

In [29]:
borough_neighborhood_venues.head()

,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,North Toronto,Willowdale,43.71174,-79.57941,Subway,43.713119,-79.580191,Sandwich Place
1,North Toronto,Willowdale,43.71174,-79.57941,Tim Hortons,43.713177,-79.580186,Coffee Shop
2,North Toronto,Willowdale,43.71174,-79.57941,Petro-Canada,43.714398,-79.581868,Gas Station
3,North Toronto,Willowdale,43.71174,-79.57941,Harvey's,43.712146,-79.577358,Restaurant
4,North Toronto,Willowdale,43.71174,-79.57941,Rexdale Hyundai,43.713733,-79.578123,Auto Dealership


In [30]:
borough_neighborhood_venues.shape

(152, 8)

#### Check how many venues been returned by each neighbour and borough

In [31]:
borough_neighborhood_venues.groupby(['Neighborhood','Borough']).count()

,,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,Borough,,,,,,
Bay Street,Downtown Toronto,16,16,16,16,16,16
Cabbagetown,Downtown Toronto,8,8,8,8,8,8
Church and Wellesley,Downtown Toronto,8,8,8,8,8,8
Design Exchange,Downtown Toronto,8,8,8,8,8,8
Grange Park,Downtown Toronto,8,8,8,8,8,8
Harbourfront,Downtown Toronto,8,8,8,8,8,8
Island airport,Downtown Toronto,8,8,8,8,8,8
King,Downtown Toronto,8,8,8,8,8,8
Rosedale,Downtown Toronto,16,16,16,16,16,16


#### To find out how many unique categories are there in returned venues

In [32]:
print(f'There are {borough_neighborhood_venues["Venue Category"].nunique()} uniques categories.')

There are 8 uniques categories.


##### Analyze each neighborhood and venue


In [33]:
# one hot encoding
toronto_onehot = pd.get_dummies(borough_neighborhood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Borough'] = borough_neighborhood_venues['Borough'] 
toronto_onehot['Neighborhoods'] = borough_neighborhood_venues['Neighborhood'] 
toronto_onehot['Venue'] = borough_neighborhood_venues['Venue'] 

# move neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]



toronto_onehot.head()

,Borough,Neighborhoods,Venue,Auto Dealership,Coffee Shop,Gas Station,Gym,Middle Eastern Restaurant,Restaurant,Sandwich Place,Storage Facility
0,North Toronto,Willowdale,Subway,0,0,0,0,0,0,1,0
1,North Toronto,Willowdale,Tim Hortons,0,1,0,0,0,0,0,0
2,North Toronto,Willowdale,Petro-Canada,0,0,1,0,0,0,0,0
3,North Toronto,Willowdale,Harvey's,0,0,0,0,0,1,0,0
4,North Toronto,Willowdale,Rexdale Hyundai,1,0,0,0,0,0,0,0


In [34]:
toronto_onehot.shape

(152, 11)

##### Now , we'll group rows by neighborhood and borough and will take the mean of the frequency of  occurence of each category

In [35]:
toronto_grouped=toronto_onehot.groupby('Neighborhoods').mean().reset_index()
toronto_grouped

,Neighborhoods,Auto Dealership,Coffee Shop,Gas Station,Gym,Middle Eastern Restaurant,Restaurant,Sandwich Place,Storage Facility
0,Bay Street,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125
1,Cabbagetown,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125
2,Church and Wellesley,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125
3,Design Exchange,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125
4,Grange Park,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125
5,Harbourfront,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125
6,Island airport,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125
7,King,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125
8,Rosedale,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125
9,Ryerson,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125


In [36]:
toronto_grouped.shape

(17, 9)

### Printing each neighborhood with top 5  most common venues 

In [37]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhoods']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhoods'] == hood].T.reset_index() #T function is to transpose
    temp.columns = ['venue','freq']
    temp = temp.iloc[2:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
    

----Bay Street----
                       venue  freq
0                Coffee Shop  0.12
1                Gas Station  0.12
2                        Gym  0.12
3  Middle Eastern Restaurant  0.12
4                 Restaurant  0.12


----Cabbagetown----
                       venue  freq
0                Coffee Shop  0.12
1                Gas Station  0.12
2                        Gym  0.12
3  Middle Eastern Restaurant  0.12
4                 Restaurant  0.12


----Church and Wellesley----
                       venue  freq
0                Coffee Shop  0.12
1                Gas Station  0.12
2                        Gym  0.12
3  Middle Eastern Restaurant  0.12
4                 Restaurant  0.12


----Design Exchange----
                       venue  freq
0                Coffee Shop  0.12
1                Gas Station  0.12
2                        Gym  0.12
3  Middle Eastern Restaurant  0.12
4                 Restaurant  0.12


----Grange Park----
                       venue  freq
0    

##### Writing the function to sort the venues in descending order

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[2:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

##### Creating the new dataframe and displaying top 7 venues for each neighborhood

In [49]:
num_top_venues = 7
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.shape
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,Bay Street,Coffee Shop,Gas Station,Gym,Middle Eastern Restaurant,Restaurant,Sandwich Place,Storage Facility
1,Cabbagetown,Coffee Shop,Gas Station,Gym,Middle Eastern Restaurant,Restaurant,Sandwich Place,Storage Facility
2,Church and Wellesley,Coffee Shop,Gas Station,Gym,Middle Eastern Restaurant,Restaurant,Sandwich Place,Storage Facility
3,Design Exchange,Coffee Shop,Gas Station,Gym,Middle Eastern Restaurant,Restaurant,Sandwich Place,Storage Facility
4,Grange Park,Coffee Shop,Gas Station,Gym,Middle Eastern Restaurant,Restaurant,Sandwich Place,Storage Facility


### Clustering the Neighborhoods

In [48]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop([ 'Neighborhoods'], 1)

# run k-means clustering
kmeans = KMeans(init='k-means++', n_clusters=kclusters, random_state=1, max_iter=1000).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:16]

<ipython-input-48-67e5cb6ff824>:7: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  kmeans = KMeans(init='k-means++', n_clusters=kclusters, random_state=1, max_iter=1000).fit(toronto_grouped_clustering)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

#### Creating a new dataframe that contains clusters as well as top 7 venues of each neighborhood

In [53]:
# add clustering labels
neighborhoods_venues_sorted['Cluster Labels'] = kmeans.labels_

toronto_merged = df_borough.copy()

# merge toronto_grouped with df_toronto_borough to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postalcode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,Cluster Labels
0,M4R,North Toronto,Willowdale,43.71174,-79.57941,Coffee Shop,Gas Station,Gym,Middle Eastern Restaurant,Restaurant,Sandwich Place,Storage Facility,0
1,M4W,Downtown Toronto,Rosedale,43.71174,-79.57941,Coffee Shop,Gas Station,Gym,Middle Eastern Restaurant,Restaurant,Sandwich Place,Storage Facility,0
2,M4X,Downtown Toronto,Cabbagetown,43.71174,-79.57941,Coffee Shop,Gas Station,Gym,Middle Eastern Restaurant,Restaurant,Sandwich Place,Storage Facility,0
3,M4Y,Downtown Toronto,Church and Wellesley,43.71174,-79.57941,Coffee Shop,Gas Station,Gym,Middle Eastern Restaurant,Restaurant,Sandwich Place,Storage Facility,0
4,M5A,Downtown Toronto,Harbourfront,43.71174,-79.57941,Coffee Shop,Gas Station,Gym,Middle Eastern Restaurant,Restaurant,Sandwich Place,Storage Facility,0


### visualize this using map

In [56]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, neighborhood,cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'],toronto_merged['Neighborhood'],toronto_merged['Cluster Labels']):
    label =  folium.Popup(str(neighborhood) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.5,
        parse_html=False).add_to(map_clusters)
map_clusters

#### Examining the clusters

In [59]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,Cluster Labels
0,North Toronto,Coffee Shop,Gas Station,Gym,Middle Eastern Restaurant,Restaurant,Sandwich Place,Storage Facility,0
1,Downtown Toronto,Coffee Shop,Gas Station,Gym,Middle Eastern Restaurant,Restaurant,Sandwich Place,Storage Facility,0
2,Downtown Toronto,Coffee Shop,Gas Station,Gym,Middle Eastern Restaurant,Restaurant,Sandwich Place,Storage Facility,0
3,Downtown Toronto,Coffee Shop,Gas Station,Gym,Middle Eastern Restaurant,Restaurant,Sandwich Place,Storage Facility,0
4,Downtown Toronto,Coffee Shop,Gas Station,Gym,Middle Eastern Restaurant,Restaurant,Sandwich Place,Storage Facility,0
5,Downtown Toronto,Coffee Shop,Gas Station,Gym,Middle Eastern Restaurant,Restaurant,Sandwich Place,Storage Facility,0
6,Downtown Toronto,Coffee Shop,Gas Station,Gym,Middle Eastern Restaurant,Restaurant,Sandwich Place,Storage Facility,0
7,Downtown Toronto,Coffee Shop,Gas Station,Gym,Middle Eastern Restaurant,Restaurant,Sandwich Place,Storage Facility,0
8,Downtown Toronto,Coffee Shop,Gas Station,Gym,Middle Eastern Restaurant,Restaurant,Sandwich Place,Storage Facility,0
9,Downtown Toronto,Coffee Shop,Gas Station,Gym,Middle Eastern Restaurant,Restaurant,Sandwich Place,Storage Facility,0
